In [21]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

# Create Ollama Chat Model
from langchain_ollama import ChatOllama
qwen_chat = ChatOllama(base_url="http://localhost:11434",
                        model="qwen3:8b",
                        temperature=0,
                        max_tokens=100)
llama_chat = ChatOllama(base_url="http://localhost:11434",
                        model="llama3.2:latest",
                        temperature=0,
                        max_tokens=100)


#### Define sample Documents

In [23]:
# Define some Documents
from langchain_core.documents import Document
docs = [
    Document(page_content="Playwright is a modern automation library for end-to-end testing. It supports multiple browsers like Chromium, Firefox, and WebKit."),
    Document(page_content="Selenium is a widely used open-source framework for web automation, supporting multiple programming languages and browsers."),
    Document(page_content="Cypress is a JavaScript-based testing tool primarily used for front-end testing. It runs in the browser and provides fast feedback loops."),
    Document(page_content="Playwright allows network interception, headless execution, and tracing for debugging complex web applications."),
    Document(page_content="Selenium WebDriver enables automated browser testing using various bindings such as Python, Java, and C#."),
    Document(page_content="Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages."),
    Document(page_content="This document talks about REST API testing tools, which are unrelated to Playwright, Selenium, or Cypress.")
]

questions = [
    "What is Playwright and what browsers does it support?",
    "What is Selenium and what programming languages does it support?",
    "What is Cypress, and how is it different from Selenium?",
    "How does Playwright handle network interception and debugging?",
    "What are the key features of Selenium WebDriver?",
    "What are the main advantages of Cypress for testing?",
    "What are REST API testing tools, and how are they different from browser automation tools?"
]

#### Create Vector Store and Retriever

In [24]:

# Create Ollama Embeddings
from langchain_ollama import OllamaEmbeddings
ollama_embeddings = OllamaEmbeddings(model="llama3.2:latest")

# Create Vector Storage
from langchain_chroma import Chroma
vector_store = Chroma.from_documents(documents=docs, 
                                     embedding=ollama_embeddings)
# Create Retriever
retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 3}
)
# Retrieve Documents
retrieved_docs = retriever.invoke("What is playwright")
retrieved_docs

[Document(id='0b2117ce-14db-4930-939c-0c2e4d0b14ef', metadata={}, page_content='Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages.'),
 Document(id='87988330-0511-4f66-966e-4433fdcc7afb', metadata={}, page_content='Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages.'),
 Document(id='1fc2b16c-87ef-46f5-810f-45ad0be4ce4d', metadata={}, page_content='Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages.')]

#### Create QA RAG Chain

In [25]:
### Create a QA Chain to invoke
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
system_prompt = """
"You are an assistance for question-answering task"
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "Use three sentences maximum to answer the question and keep the answer concise."
    "The answer should be in bullet points."
    "\n\n{context}\n\n"
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

def format_doc(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {
        "context": retriever | format_doc,
        "question": RunnablePassthrough()
    } 
    | prompt_template 
    | llama_chat
    | StrOutputParser()
)

In [26]:
response = rag_chain.invoke("What is PlayWright")
print(response)

* Playwright is a browser automation framework that allows developers to automate complex web applications.
* It provides features such as network interception, headless execution, and tracing for debugging purposes.
* With Playwright, developers can write tests for their web applications in a more efficient and reliable way.


#### Creating Mutlishot Data Set for RAGAs

In [ ]:
dataset = []

for question, doc in zip(questions, docs):
    relevant_docs = [doc.page_content for doc in retriever.invoke(question)]
    response = rag_chain.invoke(question)
    dataset.append(
        {
            "user_input": question,
            "retrieved_contexts": relevant_docs,
            "response": response,
            "reference": doc.page_content
        }
    )
dataset

#### Evaluation Using RAGAs

In [29]:
from ragas import EvaluationDataset
from ragas import evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.run_config import RunConfig

run_config = RunConfig(timeout=300, max_workers=1)
evaluation_dataset = EvaluationDataset.from_list(dataset)
result = evaluate(dataset= evaluation_dataset,
                  metrics=[Faithfulness()],
                  llm= qwen_chat, 
                  run_config=run_config)
result

Evaluating: 100%|██████████| 7/7 [16:16<00:00, 139.47s/it]


{'faithfulness': 0.3313}

In [30]:
result.to_pandas()

,user_input,retrieved_contexts,response,reference,faithfulness
0,What is Playwright and what browsers does it s...,[Playwright is a modern automation library for...,Here are the key points about Playwright:\n\n*...,Playwright is a modern automation library for ...,1.000000
1,What is Selenium and what programming language...,[Selenium WebDriver enables automated browser ...,Here are the key points about Selenium:\n\n* S...,Selenium is a widely used open-source framewor...,0.375000
2,"What is Cypress, and how is it different from ...",[Selenium WebDriver enables automated browser ...,Here are the key differences between Cypress a...,Cypress is a JavaScript-based testing tool pri...,0.200000
3,How does Playwright handle network interceptio...,"[Playwright allows network interception, headl...",* Playwright allows network interception throu...,"Playwright allows network interception, headle...",0.285714
4,What are the key features of Selenium WebDriver?,[Selenium WebDriver enables automated browser ...,Here are the key features of Selenium WebDrive...,Selenium WebDriver enables automated browser t...,0.083333
5,What are the main advantages of Cypress for te...,[Cypress is a JavaScript-based testing tool pr...,Here are the main advantages of Cypress for te...,"Cypress has built-in support for retries, time...",0.375000
6,"What are REST API testing tools, and how are t...",[This document talks about REST API testing to...,* REST API testing tools are software applicat...,This document talks about REST API testing too...,0.000000
